In [1]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
# from datapackage import Package
# from selenium import webdriver
# import requests
# from bs4 import BeautifulSoup
# import yfinance as yf
import os
import random
import shutil
import time
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from tensorflow.keras.utils import plot_model
import graphviz
import pydot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [2]:
# PARAMETERS

steps = 250
epochs = 50
nodes_per_layer = 64
shift = -1
batch_size = 64
loss_function = "mean_squared_error"
dropout_rate = 0.05

In [ ]:
start_time = time.time()

for file in os.listdir("data"):
	ticker_code = file.replace(".csv", "")
	dataset = pd.read_csv("data/" + file)
	shift = -1
	dataset = dataset.drop(columns=["Date", "Dividends", "Stock Splits"])
	train_set_len = int(len(dataset) * 0.8)
	train_set = dataset[:train_set_len]
	test_set = dataset[train_set_len:]
	X_train = train_set.drop(columns=["Close"])[:shift]
	y_train = train_set["Close"].shift(shift).dropna()
	X_test = test_set.drop(columns=["Close"])[:shift]
	y_test = test_set["Close"].shift(shift).dropna()
	x_scaler = MinMaxScaler()
	y_scaler = MinMaxScaler()
	X_train_scaled = x_scaler.fit_transform(X_train)
	y_train_scaled = y_scaler.fit_transform(np.array(y_train).reshape(-1,1))
	X_test_scaled = x_scaler.transform(X_test)
	y_test_scaled = y_scaler.transform(np.array(y_test).reshape(-1,1))
	pd.to_pickle(train_set, "train_sets/" + ticker_code + "_train_set.pkl")
	pd.to_pickle(test_set, "test_sets/" + ticker_code + "_test_set.pkl")
	pd.to_pickle(X_train, "sets/X_train/" + ticker_code + "_X_train.pkl")
	pd.to_pickle(X_test, "sets/X_test/" + ticker_code + "_X_test.pkl")
	pd.to_pickle(y_train, "sets/y_train/" + ticker_code + "_y_train.pkl")
	pd.to_pickle(y_test, "sets/y_test/" + ticker_code + "_y_test.pkl")
	pd.to_pickle(X_train_scaled, "sets/X_train/" + ticker_code + "_X_train_scaled.pkl")
	pd.to_pickle(X_test_scaled, "sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
	pd.to_pickle(y_train_scaled, "sets/y_train/" + ticker_code + "_y_train_scaled.pkl")
	pd.to_pickle(y_test_scaled, "sets/y_test/" + ticker_code + "_y_test_scaled.pkl")


In [3]:
# old_top_25 = ["AAPL", "MSFT", "AMZN", "TSLA", "GOOGL", "BRK.B", "UNH", "JNJ", "XOM", "JPM", "META", "V", "PG", "NVDA", "HD", "CVX", "LLY", "MA", "ABBV", "PFE", "MRK", "PEP", "BAC", "KO"]
start_time = time.time()
top_25 = ["ABBV", "BRK.B", "CVX", "GOOGL", "MRK"]
for ticker_code in top_25:
	print(ticker_code)
	dataset = pd.read_csv("data/" + ticker_code + ".csv")
	non_norm_features = dataset.drop(columns = ["Date", "Dividends", "Stock Splits", "Close"])
	labels = dataset["Close"].shift(-1)
	scaler = MinMaxScaler()
	features = scaler.fit_transform(non_norm_features)
	X_train_list = []
	for i in range(steps, len(features)):
		X_train_list.append(features[i-steps:i])
	X_train_list = np.array(X_train_list)
	model = Sequential()
	model.add(LSTM(nodes_per_layer, activation='relu', input_shape=(steps, X_train_list.shape[2]), return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=False))
	model.add(Dense(units=1, activation='relu'))
	model.compile(optimizer='adam', loss=loss_function)
	model.fit(X_train_list, labels[:(steps * shift)], epochs=epochs, batch_size=batch_size)
	X_test_scaled = pd.read_pickle("sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
	y_test_scaled = pd.read_pickle("sets/y_test/" + ticker_code + "_y_test.pkl")
	X_test_list = []
	for i in range(steps, len(X_test_scaled)):
		X_test_list.append(X_test_scaled[i-steps:i])
	X_test_list = np.array(X_test_list)
	score = model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])
	print(f"{ticker_code} processing finished")
	pd.to_pickle(model, "models/" + ticker_code + "_score_" + str(round(score, 2)) + ".pkl")

ABBV
Epoch 1/50
36/36 [==============================] - 52s 1s/step - loss: 4245.1313
Epoch 2/50
36/36 [==============================] - 35s 975ms/step - loss: 4245.1299
Epoch 3/50
36/36 [==============================] - 42s 1s/step - loss: 4245.1309
Epoch 4/50
36/36 [==============================] - 40s 1s/step - loss: 4245.1304
Epoch 5/50
36/36 [==============================] - 42s 1s/step - loss: 4245.1304
Epoch 6/50
36/36 [==============================] - 46s 1s/step - loss: 4245.1299
Epoch 7/50
36/36 [==============================] - 46s 1s/step - loss: 4245.1299
Epoch 8/50
36/36 [==============================] - 45s 1s/step - loss: 4245.1309
Epoch 9/50
36/36 [==============================] - 42s 1s/step - loss: 4245.1309
Epoch 10/50
36/36 [==============================] - 33s 896ms/step - loss: 4245.1309
Epoch 11/50
36/36 [==============================] - 37s 1s/step - loss: 4245.1304
Epoch 12/50
36/36 [==============================] - 36s 1s/step - loss: 4245.1304
Ep

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_9884\421977027.py:38: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  score = model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])


8/8 [==============================] - 4s 270ms/step - loss: 10984.6533
ABBV processing finished
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\dropout_3
......vars
...layers\dropout_4
......vars
...layers\dropout_5
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...layers\lstm_3
......vars
...layers\lstm_3\cell
......vars
.........0
.........1
.........2
...layers\lstm_4
......vars
...layers\lstm_4\cell
......vars
.........0
.........1
.........2
...layers\lstm_5
......vars
...layers\lstm_5\cell
......vars
.........0
.........1
.........2
...layers\lstm_6
......vars
...layers\lstm_6

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_9884\421977027.py:38: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  score = model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])


34/34 [==============================] - 11s 276ms/step - loss: nan
BRK.B processing finished
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\dropout_3
......vars
...layers\dropout_4
......vars
...layers\dropout_5
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...layers\lstm_3
......vars
...layers\lstm_3\cell
......vars
.........0
.........1
.........2
...layers\lstm_4
......vars
...layers\lstm_4\cell
......vars
.........0
.........1
.........2
...layers\lstm_5
......vars
...layers\lstm_5\cell
......vars
.........0
.........1
.........2
...layers\lstm_6
......vars
...layers\lstm_6\ce

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_9884\421977027.py:38: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  score = model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])


88/88 [==============================] - 35s 370ms/step - loss: nan
CVX processing finished
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\dropout_3
......vars
...layers\dropout_4
......vars
...layers\dropout_5
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...layers\lstm_3
......vars
...layers\lstm_3\cell
......vars
.........0
.........1
.........2
...layers\lstm_4
......vars
...layers\lstm_4\cell
......vars
.........0
.........1
.........2
...layers\lstm_5
......vars
...layers\lstm_5\cell
......vars
.........0
.........1
.........2
...layers\lstm_6
......vars
...layers\lstm_6\cell

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_9884\421977027.py:38: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  score = model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])


21/21 [==============================] - 11s 411ms/step - loss: 7265.8599
GOOGL processing finished
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\dropout_3
......vars
...layers\dropout_4
......vars
...layers\dropout_5
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...layers\lstm_3
......vars
...layers\lstm_3\cell
......vars
.........0
.........1
.........2
...layers\lstm_4
......vars
...layers\lstm_4\cell
......vars
.........0
.........1
.........2
...layers\lstm_5
......vars
...layers\lstm_5\cell
......vars
.........0
.........1
.........2
...layers\lstm_6
......vars
...layers\lst

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(features).isna().sum()